In [2]:
import math
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from tqdm.notebook import tqdm
import random

import numba as nb
# from numba import jit, vectorize, float64

In [8]:
with open('./data/tsp_51_1', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')
nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    # points.append(Point(float(parts[0]), float(parts[1])))
    points.append((float(parts[0]), float(parts[1])))
points = np.array(points)

In [76]:
import random
import math
import time

# Define structures
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Connection:
    def __init__(self):
        self.in_node = None
        self.out_node = None

class Penalty:
    def __init__(self, node_count):
        self.penalty = [[0] * i for i in range(1, node_count + 1)]

    def __getitem__(self, key):
        j, i = sorted(key)
        return self.penalty[i][j]

    def __setitem__(self, key, value):
        j, i = sorted(key)
        self.penalty[i][j] = value

class Activate:
    def __init__(self, size):
        self.bits = [1] * size
        self.ones = size

    def set_1(self, i):
        if self.bits[i] == 0:
            self.ones += 1
        self.bits[i] = 1

    def set_0(self, i):
        if self.bits[i] == 1:
            self.ones -= 1
        self.bits[i] = 0

    def get(self, i):
        return self.bits[i]

    def size(self):
        return len(self.bits)

def print_tour(connection):
    node = 0
    for i in range(len(connection)):
        print(node, end=' ')
        node = connection[node].out_node
    print()

def get_distance_matrix(node_vec):
    def square(x):
        return x * x

    def distance(a, b):
        return math.sqrt(square(a.x - b.x) + square(a.y - b.y))

    def distance_matrix(i, j):
        return distance(node_vec[i], node_vec[j])

    return distance_matrix

def init_connection(node_count, distance_matrix):
    tour = list(range(node_count))
    
    for i in range(len(tour) - 1):
        min_distance = float('inf')
        min_distance_node = -1

        for j in range(i + 1, len(tour)):
            dist = distance_matrix(tour[i], tour[j])
            if min_distance > dist:
                min_distance = dist
                min_distance_node = j

        tour[i + 1], tour[min_distance_node] = tour[min_distance_node], tour[i + 1]

    connection = [Connection() for _ in range(node_count)]
    for i in range(len(tour)):
        node = tour[i]
        next_node = tour[(i + 1) % len(tour)]
        connection[node].out_node = next_node
        connection[next_node].in_node = node
    print(tour)
    return connection

def random_sample(vec):
    assert len(vec) > 0
    random_index = random.randint(0, len(vec) - 1)
    return vec[random_index]

def select_t3_t4(t1, t2, connection, distance_matrix, penalty, lambda_val):
    max_gain = float('-inf')
    t4_candidate = []
    t2_out = connection[t2].out_node

    for i in range(len(connection)):
        t4 = i
        t3 = connection[t4].in_node

        if t4 in (t1, t2, t2_out):
            continue

        d12 = distance_matrix(t1, t2)
        d34 = distance_matrix(t3, t4)
        d13 = distance_matrix(t1, t3)
        d24 = distance_matrix(t2, t4)

        p12 = penalty[t1,t2]
        p34 = penalty[t3,t4]
        p13 = penalty[t1,t3]
        p24 = penalty[t2,t4]

        gain = (d13 + lambda_val * p13) + (d24 + lambda_val * p24) - (d12 + lambda_val * p12) - (d34 + lambda_val * p34)

        if gain > max_gain:
            max_gain = gain
            t4_candidate = [t4]
        elif gain == max_gain:
            t4_candidate.append(t4)
    print(max_gain)
    if max_gain > 1e-6:
        t4 = random_sample(t4_candidate)
        t3 = connection[t4].in_node
        print(t1, t2, t3, t4)
        return t3, t4

    return -1, -1

def swap_edge(t1, t2, t3, t4, connection, distance_matrix, penalty, distance, augmented_distance, lambda_val):
    cur_node = t2
    cur_node_out = connection[cur_node].out_node

    while cur_node != t3:
        next_cur_node = cur_node_out
        next_cur_node_out = connection[next_cur_node].out_node

        connection[cur_node].in_node = cur_node_out
        connection[cur_node_out].out_node = cur_node

        cur_node = next_cur_node
        cur_node_out = next_cur_node_out

    connection[t2].out_node = t4
    connection[t4].in_node = t2

    connection[t1].out_node = t3
    connection[t3].in_node = t1

    d12 = distance_matrix(t1, t2)
    d34 = distance_matrix(t3, t4)
    d13 = distance_matrix(t1, t3)
    d24 = distance_matrix(t2, t4)

    p12 = penalty[t1, t2]
    p34 = penalty[t3, t4]
    p13 = penalty[t1, t3]
    p24 = penalty[t2, t4]

    # gain = (d12 + lambda_val * p12) + (d34 + lambda_val * p34) - (d13 + lambda_val * p13) - (d24 + lambda_val * p24)
    gain = (d13 + lambda_val * p13) + (d24 + lambda_val * p24) - (d12 + lambda_val * p12) - (d34 + lambda_val * p34)

    distance -= d12 + d34 - d13 - d24
    augmented_distance -= gain

    return distance, augmented_distance

def add_penalty(connection, distance_matrix, penalty, activate, augmented_distance, lambda_val):
    max_util = float('-inf')
    max_util_node = []

    for i in range(len(connection)):
        i_out = connection[i].out_node
        d = distance_matrix(i, i_out)
        p = (1 + penalty[i, i_out])
        util = d / (1 + p)

        if util > max_util:
            max_util = util
            max_util_node = [i]
        elif util == max_util:
            max_util_node.append(i)

    for i in max_util_node:
        i_out = connection[i].out_node
        penalty[i, i_out] += 1

        activate.set_1(i)
        activate.set_1(i_out)

        augmented_distance += lambda_val

def total_distance(connection, distance_matrix):
    dis = 0.0
    for i in range(len(connection)):
        dis += distance_matrix(i, connection[i].out_node)

    return dis

def total_augmented_distance(connection, distance_matrix, penalty, lambda_val):
    augmented_dis = 0.0
    for i in range(len(connection)):
        i_out = connection[i].out_node
        d = distance_matrix(i, i_out)
        
        p = penalty[i, i_out]
        
        augmented_dis += d + p * lambda_val

    return augmented_dis

def save_result(filename, distance, connection):
    with open(filename, "w") as f:
        f.write(f"{distance} 0\n")
        print_tour(connection, f)

def load_node(filename):
    with open(filename, "r") as f:
        node_count = int(f.readline())
        node_vec = []

        for _ in range(node_count):
            x, y = map(float, f.readline().split())
            node_vec.append(Node(x, y))

        return node_vec

def init_lambda(connection, distance_matrix, alpha):
    return alpha * total_distance(connection, distance_matrix) / len(connection)

def search(connection, distance_matrix):
    step_limit = 1000000

    penalty = Penalty(len(connection))
    alpha = 0.1
    lambda_val = 0.0

    activate = Activate(len(connection))

    current_connection = connection
    current_distance = total_distance(current_connection, distance_matrix)
    current_augmented_distance = total_augmented_distance(current_connection, distance_matrix, penalty, lambda_val)

    best_connection = current_connection
    best_distance = current_distance

    for cur_step in range(step_limit):

        if cur_step%10 == 0:
            print(f"[step {cur_step + 1:8}] [current distance {current_distance}] [current augmented distance {current_augmented_distance}] [best distance {best_distance}]")
        
        while activate.ones > 0:
            for bit in range(activate.size()):
                if not activate.get(bit):
                    continue

                bit_in = current_connection[bit].in_node
                bit_out = current_connection[bit].out_node

                t1_t2_candidate = [(bit_in, bit), (bit, bit_out)]

                for j in range(len(t1_t2_candidate)):
                    t1, t2 = t1_t2_candidate[j]

                    t3, t4 = select_t3_t4(t1, t2, current_connection, distance_matrix, penalty, lambda_val)

                    if t3 == -1:
                        if j == 1:
                            activate.set_0(bit)
                            print('Setting zero: ', bit)
                        continue

                    current_distance, current_augmented_distance = swap_edge(t1, t2, t3, t4, current_connection, distance_matrix, penalty, current_distance, current_augmented_distance, lambda_val)
                    print(current_distance, current_augmented_distance)

                    idx = t3
                    print_path = []
                    for _ in range(51):
                        print_path.append(idx)
                        idx = connection[idx].out_node
                    print(np.array(print_path))
                    
                    activate.set_1(t1)
                    activate.set_1(t2)
                    activate.set_1(t3)
                    activate.set_1(t4)

                    break

                if best_distance > current_distance:
                    best_connection = current_connection
                    best_distance = current_distance

                    # save_result("python_output.txt", best_distance, best_connection)
            print(activate.ones)
        break

        if lambda_val == 0.0:
            lambda_val = init_lambda(connection, distance_matrix, alpha)
            
        add_penalty(current_connection, distance_matrix, penalty, activate, current_augmented_distance, lambda_val)

    # save_result("python_output.txt", best_distance, best_connection)
    # return best_connection

In [77]:
node_vec = load_node('./data/tsp_51_1')

distance_matrix = get_distance_matrix(node_vec)
connection = init_connection(len(node_vec), distance_matrix)

best_connection = search(connection, distance_matrix)
# best_distance = total_distance(best_connection, distance_matrix)

# print(f"best distance : {best_distance}")
# print("best tour:")
# print_tour(best_connection)

[0, 33, 5, 2, 28, 45, 9, 10, 3, 46, 24, 34, 23, 12, 36, 6, 26, 47, 27, 41, 8, 4, 35, 13, 7, 19, 40, 18, 11, 42, 29, 43, 21, 37, 20, 25, 1, 22, 31, 39, 50, 38, 15, 44, 16, 14, 30, 48, 32, 17, 49]
[step        1] [current distance 506.36316536284625] [current augmented distance 506.36316536284625] [best distance 506.36316536284625]
85.27284111282941
49 0 13 7
591.6360064756757 421.0903242500168
[13 35  4  8 41 27 47 26  6 36 12 23 34 24 46  3 10  9 45 28  2  5 33  0
  7 19 40 18 11 42 29 43 21 37 20 25  1 22 31 39 50 38 15 44 16 14 30 48
 32 17 49]
65.19019683682396
25 1 10 9
656.8262033124996 355.9001274131929
[10  3 46 24 34 23 12 36  6 26 47 27 41  8  4 35 13 49 17 32 48 30 14 16
 44 15 38 50 39 31 22  1  9 45 28  2  5 33  0  7 19 40 18 11 42 29 43 21
 37 20 25]
109.77155442194089
28 2 14 16
766.5977577344405 246.12857299125199
[14 30 48 32 17 49 13 35  4  8 41 27 47 26  6 36 12 23 34 24 46  3 10 25
 20 37 21 43 29 42 11 18 40 19  7  0 33  5  2 16 44 15 38 50 39 31 22  1
  9 45 28]
10

In [59]:
best_connection

In [9]:
penalty = Penalty(nodeCount)
activate = Activate(nodeCount)

In [12]:
current_connection = connection
current_distance = total_distance(current_connection, distance_matrix)
current_augmented_distance = total_augmented_distance(current_connection, distance_matrix, penalty, lambda_val)

In [13]:
current_distance, current_augmented_distance

(506.36316536284625, 506.36316536284625)

In [10]:
t1 = connection[2].in_node
t2 = 2
t2_out = connection[t2].out_node

t4 = 45
t3 = connection[t4].in_node

if t4 in (t1, t2, t2_out):
    print('ignore')

d12 = distance_matrix(t1, t2)
d34 = distance_matrix(t3, t4)
d13 = distance_matrix(t1, t3)
d24 = distance_matrix(t2, t4)

p12 = penalty[t1,t2]
p34 = penalty[t3,t4]
p13 = penalty[t1,t3]
p24 = penalty[t2,t4]

gain = (d12 + lambda_val * p12) + (d34 + lambda_val * p34) - (d13 + lambda_val * p13) - (d24 + lambda_val * p24)

In [14]:
current_distance, current_augmented_distance = swap_edge(t1, t2, t3, t4, current_connection, distance_matrix, penalty, current_distance, current_augmented_distance, lambda_val)

In [71]:
idx = 0
print_path = []
for _ in range(51):
    print_path.append(idx)
    idx = connection[idx].out_node
np.array(print_path)    

In [73]:
np.array(print_path)

array([ 0, 19, 33, 40,  5, 11, 26, 18,  2, 42, 47, 16,  6, 37, 28, 44, 10,
       14,  9, 15, 45, 29, 27, 21,  3, 38, 36, 43, 41, 50, 46, 20, 24, 39,
        8, 25, 34, 49, 12, 31,  4, 17, 23, 48, 13, 32, 35, 22, 30,  1,  7])